03717357 Gregory Ollivierre COMP6040 Assignment2 Task A & B

Apologies in advance for the semingly random imports below. 

Depending on how you installed pyspark, you may need to enable the os.environ variables and/or install findspark.


In [3]:
# from pathlib import Path
# home = str(Path.home())
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[2] pyspark-shell"
# os.environ['JAVA_HOME'] = home+"/jdk1.8.0_161/"
# import findspark
# spark_dir = home+'/spark-2.3.0-bin-hadoop2.7/'
# findspark.init(spark_dir)
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *

ModuleNotFoundError: No module named 'pyspark'

In [218]:
#A.1Convert the following sentence into a python tuple list of letters and the frequency of which each letter appears in the current word.
#Ignore all non-alpha numeric characters.
sentence = "The quick brown fox jumps over the laxy dog and the fox was very happy"
# I treated 'T' and 't' as two seperate chars since they are different. Alternatively sentence.tolower()
# could be used
samp_tup_list = [(x,sentence.count(x)) for x in set(sentence.replace(" ",''))]
samp_tup_list

[('s', 2),
 ('i', 1),
 ('y', 3),
 ('T', 1),
 ('j', 1),
 ('o', 5),
 ('c', 1),
 ('f', 2),
 ('q', 1),
 ('p', 3),
 ('l', 1),
 ('x', 3),
 ('a', 4),
 ('r', 3),
 ('e', 5),
 ('k', 1),
 ('b', 1),
 ('n', 2),
 ('h', 4),
 ('m', 1),
 ('t', 2),
 ('d', 2),
 ('u', 2),
 ('v', 2),
 ('w', 2),
 ('g', 1)]

In [219]:
#A.2 create pyspark contect
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext

In [220]:
#A.3 convert list of tuples to rdd
rdd = sc.parallelize(samp_tup_list)
rdd.first()

('s', 2)

In [221]:
#A.4 display letter count
rdd.collect()

[('s', 2),
 ('i', 1),
 ('y', 3),
 ('T', 1),
 ('j', 1),
 ('o', 5),
 ('c', 1),
 ('f', 2),
 ('q', 1),
 ('p', 3),
 ('l', 1),
 ('x', 3),
 ('a', 4),
 ('r', 3),
 ('e', 5),
 ('k', 1),
 ('b', 1),
 ('n', 2),
 ('h', 4),
 ('m', 1),
 ('t', 2),
 ('d', 2),
 ('u', 2),
 ('v', 2),
 ('w', 2),
 ('g', 1)]

In [222]:
#A.5 display the letter and number of times they appear in each word in the sentence
rdd.map(lambda x : (x[0],[y.count(x[0]) for y in sentence.split() if x[0] in y])).collect()

[('s', [1, 1]),
 ('i', [1]),
 ('y', [1, 1, 1]),
 ('T', [1]),
 ('j', [1]),
 ('o', [1, 1, 1, 1, 1]),
 ('c', [1]),
 ('f', [1, 1]),
 ('q', [1]),
 ('p', [1, 2]),
 ('l', [1]),
 ('x', [1, 1, 1]),
 ('a', [1, 1, 1, 1]),
 ('r', [1, 1, 1]),
 ('e', [1, 1, 1, 1, 1]),
 ('k', [1]),
 ('b', [1]),
 ('n', [1, 1]),
 ('h', [1, 1, 1, 1]),
 ('m', [1]),
 ('t', [1, 1]),
 ('d', [1, 1]),
 ('u', [1, 1]),
 ('v', [1, 1]),
 ('w', [1, 1]),
 ('g', [1])]

In [223]:
#B.1 create new pyspark contect
spark2 = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc2 = spark2.sparkContext

In [224]:
#B.2lets open the csv and save to rdd
rdd2 = sc2.textFile('amazon_reviews.csv')
rdd2.take(5)

[',id,name,username',
 '0,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Cristina M',
 '1,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Ricky',
 '2,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Tedd Gardiner',
 '3,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Dougal']

In [225]:
#B.3 looks like comma seperated data. Lets split by ',' and remove first line since we add schema later
rdd3 = rdd2.map(lambda line: line.split(','))
rdd3 = rdd3.filter(lambda line : line[0] != '')
rdd3.take(5)

[['0', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Cristina M'],
 ['1', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Ricky'],
 ['2', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Tedd Gardiner'],
 ['3', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Dougal'],
 ['4', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Miljan David Tanic']]

In [226]:
#B.3 create dataframe with schema (index,idn,product,username)
df = rdd3.map(lambda line: Row(index=line[0],idn=line[1],product=line[2],username=line[3])).toDF()
df = df.drop('index') # drop index since its prob not required
df.show()

+--------------------+-----------------+--------------------+
|                 idn|          product|            username|
+--------------------+-----------------+--------------------+
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|          Cristina M|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|               Ricky|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|       Tedd Gardiner|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|              Dougal|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|  Miljan David Tanic|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|          Kelvin Law|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|               Ricky|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|             Bandler|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|          Cristina M|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|       Tedd Gardiner|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|     Miguel Martinez|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|   Magnus Brattemark|
|AVpe7AsMilAPnD_xQ78G|Kindle Paperwhite|       Tedd Gardiner|
|AVpe7As

In [227]:
#B.4 display top 20 bought products
df.groupby('product').count().sort('count',ascending=False).take(20)

[Row(product='Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', count=542),
 Row(product='Amazon Fire TV', count=166),
 Row(product='Amazon Premium Headphones', count=133),
 Row(product='Fire HD 6 Tablet', count=87),
 Row(product='"Kindle Fire HDX 7"""', count=53),
 Row(product='"Kindle Fire HDX 8.9"""', count=43),
 Row(product='"Kindle Fire HD 7"""', count=41),
 Row(product='Kindle Paperwhite', count=39),
 Row(product='Certified Refurbished Amazon Fire TV (Previous Generation - 1st)', count=38),
 Row(product='Kindle Keyboard', count=32),
 Row(product='All-New Amazon Fire 7 Tablet Case (7th Generation', count=27),
 Row(product='Kindle', count=20),
 Row(product='Amazon 5W USB Official OEM Charger and Power Adapter for Fire Tablets and Kindle eReaders', count=19),
 Row(product='All-New Amazon Fire HD 8 Tablet Case (7th Generation', count=18),
 Row(product='Replacement Remote for Amazon Fire TV Stick', count=17),
 Row(product='All-New Amazon Kid-Proof Case for Amazon Fire 7 Tablet (

In [228]:
#B.5 display top 20 users and what they bought
df.groupby('username').agg(count('username'),collect_set('product'))\
                      .sort('count(username)',ascending=False).take(20)

[Row(username='A. Younan', count(username)=59, collect_set(product)=['Amazon Premium Headphones']),
 Row(username='William Hardin', count(username)=58, collect_set(product)=['Amazon Fire TV', 'Fire HD 6 Tablet', 'Certified Refurbished Amazon Fire TV (Previous Generation - 1st)']),
 Row(username='Andrew', count(username)=43, collect_set(product)=['Amazon Premium Headphones']),
 Row(username='Victor L.', count(username)=30, collect_set(product)=['Amazon Premium Headphones']),
 Row(username='Mike W.', count(username)=29, collect_set(product)=['Fire HD 6 Tablet']),
 Row(username='NF', count(username)=29, collect_set(product)=['"Kindle Fire HDX 7"""', '"Kindle Fire HD 7"""']),
 Row(username='Earthling1984', count(username)=28, collect_set(product)=['Fire HD 6 Tablet']),
 Row(username='Amazon Reviewer', count(username)=27, collect_set(product)=['"Kindle Fire HDX 7"""', '"Kindle Fire HDX 8.9"""']),
 Row(username='Mandy', count(username)=26, collect_set(product)=['Amazon Fire TV', 'Certified R

In [229]:
#B.6 Create a RDD of tuples from the dataframe from question 3 with only 2 columns ‘product’ and ‘username’ in that order.
rdd4 = df.select('product','username').rdd.map(tuple)
rdd4.take(5)

[('Kindle Paperwhite', 'Cristina M'),
 ('Kindle Paperwhite', 'Ricky'),
 ('Kindle Paperwhite', 'Tedd Gardiner'),
 ('Kindle Paperwhite', 'Dougal'),
 ('Kindle Paperwhite', 'Miljan David Tanic')]

In [230]:
#B.7 Using methods from PySpark’s RDD object e.g. groupByKey, map, reduceByKey, derive the top 20 products.
rdd4.groupByKey().map(lambda x: (x[0],len(x[1]))).sortBy(lambda x: x[1],False).take(20)

[('Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 542),
 ('Amazon Fire TV', 166),
 ('Amazon Premium Headphones', 133),
 ('Fire HD 6 Tablet', 87),
 ('"Kindle Fire HDX 7"""', 53),
 ('"Kindle Fire HDX 8.9"""', 43),
 ('"Kindle Fire HD 7"""', 41),
 ('Kindle Paperwhite', 39),
 ('Certified Refurbished Amazon Fire TV (Previous Generation - 1st)', 38),
 ('Kindle Keyboard', 32),
 ('All-New Amazon Fire 7 Tablet Case (7th Generation', 27),
 ('Kindle', 20),
 ('Amazon 5W USB Official OEM Charger and Power Adapter for Fire Tablets and Kindle eReaders',
  19),
 ('All-New Amazon Fire HD 8 Tablet Case (7th Generation', 18),
 ('Replacement Remote for Amazon Fire TV Stick', 17),
 ('All-New Amazon Kid-Proof Case for Amazon Fire 7 Tablet (7th Generation',
  13),
 ('Echo Dot (2nd Generation) - Black', 13),
 ('Alexa Voice Remote for Amazon Fire TV and Fire TV Stick', 12),
 ('All-New Amazon Kid-Proof Case for Amazon Fire HD 8 Tablet (7th Generation',
  12),
 ('Moshi Anti-Glare No Bubble Screen Protect

In [231]:
#B.8 Create a RDD of tuples from the dataframe from question 3 with only 2 columns 'username’ and ‘product’ in that order.
rdd5 = df.select('username','product').rdd.map(tuple)
rdd5.take(5)

[('Cristina M', 'Kindle Paperwhite'),
 ('Ricky', 'Kindle Paperwhite'),
 ('Tedd Gardiner', 'Kindle Paperwhite'),
 ('Dougal', 'Kindle Paperwhite'),
 ('Miljan David Tanic', 'Kindle Paperwhite')]

In [232]:
#B.9 Using methods from PySparks’s RDD object, produce the top 10 customers who purchased the most items.
#The top 10 list must show the usernames and a list of all the items each person bought. 
#Each item should have an associated quantity value representing the amount of the item which was
#purchased by the customer.

rdd5.groupByKey()\
    .map(lambda x: (x[0],dict([(y,list(x[1]).count(y)) for y in list(x[1])]),len(x[1])))\
    .sortBy(lambda x: x[2],False)\
    .take(10)

[('A. Younan', {'Amazon Premium Headphones': 59}, 59),
 ('William Hardin',
  {'Amazon Fire TV': 16,
   'Certified Refurbished Amazon Fire TV (Previous Generation - 1st)': 12,
   'Fire HD 6 Tablet': 30},
  58),
 ('Andrew', {'Amazon Premium Headphones': 43}, 43),
 ('Victor L.', {'Amazon Premium Headphones': 30}, 30),
 ('NF', {'"Kindle Fire HD 7"""': 14, '"Kindle Fire HDX 7"""': 15}, 29),
 ('Mike W.', {'Fire HD 6 Tablet': 29}, 29),
 ('Earthling1984', {'Fire HD 6 Tablet': 28}, 28),
 ('Amazon Reviewer',
  {'"Kindle Fire HDX 7"""': 13, '"Kindle Fire HDX 8.9"""': 14},
  27),
 ('Mandy',
  {'Amazon Fire TV': 16,
   'Certified Refurbished Amazon Fire TV (Previous Generation - 1st)': 10},
  26),
 ('Amazon Customer',
  {'"Kindle Fire HD 7"""': 1,
   '"Kindle Fire HDX 8.9"""': 1,
   'Alexa Voice Remote for Amazon Fire TV and Fire TV Stick': 3,
   'All-New Amazon Fire 7 Tablet Case (7th Generation': 2,
   'All-New Amazon Kid-Proof Case for Amazon Fire 7 Tablet (7th Generation': 2,
   'All-New Fire 7